In [1]:
from langame import LangameClient
import numpy as np
from autofaiss import build_index
import os
import shutil
import numpy as np
from autofaiss import build_index
from sentence_transformers import SentenceTransformer
import torch
from langame.quality import is_garbage
from torch.utils.data import DataLoader
from tqdm import tqdm

import os
c = LangameClient(path_to_config_file="../config.yaml")

In [15]:
firestore = c._firestore_client
collection = firestore.collection("memes").limit(10)
existing_conversation_starters = []

for e in collection.stream():
    if is_garbage(e.to_dict()):
        print(f"Skipping id: {e.id}, garbage, data: {e.to_dict()},")
        continue
    existing_conversation_starters.append({"id": e.id, **e.to_dict()})
print(
    f"Got {len(existing_conversation_starters)} existing conversation starters"
)
print("Preparing embeddings for existing conversation starters")
sentence_embeddings_model = None

sentence_embeddings_model_name = "sentence-transformers/LaBSE"
device = "cpu"

print(f"Loaded sentence embedding model, device: {device}")

sentence_embeddings_model = SentenceTransformer(
    sentence_embeddings_model_name, device=device
)

Got 10 existing conversation starters
Preparing embeddings for existing conversation starters
Loaded sentence embedding model, device: cpu


ERROR: Unexpected segmentation fault encountered in worker.
 

In [16]:
bar = tqdm()
embeddings = []
for item in existing_conversation_starters:
    emb = np.array(sentence_embeddings_model.encode(
        item["content"], show_progress_bar=False, device=device
    ))

    item["embedding"] = emb
    embeddings.append(emb)
    bar.update(1)

10it [02:26, 14.68s/it]


In [17]:
# delete "embeddings" and "indexes" folders
for folder in ["embeddings", "indexes"]:
    if os.path.exists(folder):
        shutil.rmtree(folder)

print("Saving embeddings to disk and building index to disk")
os.makedirs("embeddings", exist_ok=True)
np.save("embeddings/p1.npy", embeddings)
index, _ = build_index(
    "embeddings",
    index_path="indexes/knn.index",
    max_index_memory_usage="6G",
    current_memory_available="7G",
)

Saving embeddings to disk and building index to disk
Using 16 omp threads (processes), consider increasing --nb_cores if you have more
Launching the whole pipeline 01/29/2022, 10:01:39
There are 10 embeddings of dim 768
	Compute estimated construction time of the index 01/29/2022, 10:01:39
		-> Train: 16.7 minutes
		-> Add: 0.0 seconds
		Total: 16.7 minutes
	>>> Finished "Compute estimated construction time of the index" in 0.0001 secs
	Checking that your have enough memory available to create the index 01/29/2022, 10:01:39
36.0KB of memory will be needed to build the index (more might be used if you have more)
	>>> Finished "Checking that your have enough memory available to create the index" in 0.0004 secs
	Selecting most promising index types given data characteristics 01/29/2022, 10:01:39
	>>> Finished "Selecting most promising index types given data characteristics" in 0.0000 secs
	Creating the index 01/29/2022, 10:01:39
		-> Instanciate the index Flat 01/29/2022, 10:01:39
		>>> F

100%|██████████| 1/1 [00:00<00:00, 2240.55it/s]

		>>> Finished "-> Adding the vectors to the index" in 0.0027 secs
	>>> Finished "Creating the index" in 0.0048 secs
	Computing best hyperparameters 01/29/2022, 10:01:39
	>>> Finished "Computing best hyperparameters" in 0.0000 secs
The best hyperparameters are: 
	Compute fast metrics 01/29/2022, 10:01:39
2000
	>>> Finished "Compute fast metrics" in 0.0259 secs
	Saving the index on local disk 01/29/2022, 10:01:39
	>>> Finished "Saving the index on local disk" in 0.0005 secs
Recap:
{'99p_search_speed_ms': 0.014972963836044071,
 'avg_search_speed_ms': 0.011739585548639297,
 'compression ratio': 0.9985372988785958,
 'index_key': 'Flat',
 'index_param': '',
 'nb vectors': 10,
 'reconstruction error %': 0.0,
 'size in bytes': 30765,
 'vectors dimension': 768}
>>> Finished "Launching the whole pipeline" in 0.0398 secs



Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/louis/Documents/langame-worker/env/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/louis/Documents/langame-worker/env/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/louis/Documents/langame-worker/env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "/home/louis/Documents/langame-worker/env/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    self._run_once()
  File "/usr/lib/python3.8/asyncio/base_even

In [55]:
query = sentence_embeddings_model.encode("intelligence", show_progress_bar=False)
_, I = index.search(np.array([query]), 20)
memes = [memes_with_embedding[i][1] for i in I[0]]

In [56]:
list(set([e["content"] for e in memes]))

['What is the mind?',
 'Is artificial intelligence itself a species from the universe?',
 'Would it be better to know less or more?',
 'Do you think that there is a unique "recipe" for knowledge?',
 'Do you think humans are the only intelligent life in the universe?',
 'What do you know now that you wish you knew when you were younger?',
 'Is it a possibility that the state become governed by an artificial intelligence?',
 'What do you feel about consciousness, what does it mean to you?',
 'Can science achieve its goal to give an explanation for everything?',
 'Do you think technology will ever be able to "understand" (or model?) the human mind?',
 'Does social status affect your intelligence?',
 'Does mysticism play a role in the study of science ?',
 'What if everything we knew was a lie?',
 'What do you think about the "knowledge economy"?',
 'How does knowledge affect our perception of the world?',
 'What is knowledge?',
 'What is your definition of intelligence?',
 "Is human intel